In [1]:
from functions import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

In [2]:
url_1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt'
url_2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt'
url_3 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt'
url_4 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt'

In [3]:
client_profile = pd.read_csv(url_1)
exp_client = pd.read_csv(url_2)

In [4]:
df_1 = pd.read_csv(url_3, parse_dates=['date_time'])
df_2 = pd.read_csv(url_4, parse_dates=['date_time'])

In [5]:
# Concat Digital Footprints (df_final_web_data) pt_1 and pt_2 into one file:
df = pd.concat([df_1, df_2])

# Remove duplicated rows in merged digital footprints file 'df'
# df.duplicated().sum() - there are 10764 duplicated rows
df.drop_duplicates(inplace=True)

In [6]:
# Merge client profiles file (df_final_demo.txt) with experiment roster (df_final_experiment_clients.txt)

clients = pd.merge(client_profile, exp_client, on='client_id', how='outer')

In [7]:
# Cleaning 'clients' file:
# 1. Fill rows wiht NaN value in 'Variation' column with 'Uknown' - 20109 rows
# 2. Drop row with NaN value in 'gender' column - 12 rows
# 3. Drop rows with 'X' value in 'gender column' - 2 rows
# 4. Drop row with NaN value in 'age' column - 1 row

#1
#clients = clients.dropna(subset=['Variation']) 
clients['Variation'] = clients['Variation'].fillna('Unknown')

#2
clients = clients.dropna(subset=['gendr']) 

#3
clients.drop(clients[clients['gendr'] == 'X'].index, inplace=True)

#4
clients = clients.dropna(subset=['clnt_age']) 

In [26]:
# Call functions to calculate average step duration, nr of clients who confirmed, nr of visits, nr of confirmations per client, errors

df_avg_step_durations = calc_avg_step_duration(df)

clients_who_confirmed = clients_who_finished(df)
clients_who_confirmed = clients_who_confirmed.set_index('client_id').sort_values(by='client_id')

df_client_visits = nr_of_visits(df)

df_client_error = detect_process_errors(df)

df_client_confirms = nr_of_confirms(df)

In [35]:
# Call funtion 'data_summary' to create data frame with web data summary

dfs =[df_client_visits, clients_who_confirmed, df_avg_step_durations, df_client_confirms, df_client_error]

web_data_summary = data_summary(dfs)

In [28]:
# df_avg_step_durations

In [27]:
# clients_who_confirmed

In [29]:
# df_client_visits

In [30]:
# df_client_error

In [31]:
# df_client_confirms

In [15]:
web_data_summary

,no_of_visits,is_confirmed,step_duration,no_of_confirms,had_error,error_count
client_id,,,,,,
169,1,True,0 days 00:00:53.250000,1,False,0
336,1,False,0 days 00:15:48,0,False,0
546,1,True,0 days 00:00:33.250000,1,False,0
555,1,True,0 days 00:00:39.500000,1,False,0
647,1,True,0 days 00:01:34.250000,1,False,0
...,...,...,...,...,...,...
9999729,3,True,0 days 00:01:07.500000,1,True,2
9999768,1,True,0 days 00:00:44.181818181,1,True,3
9999832,1,False,0 days 00:00:08,0,False,0


In [36]:
# Merge web_data_summary with 'clients' data frame

all = pd.merge(clients, web_data_summary, on='client_id', how='outer')

# Drop rows with NaN values
all = all.dropna()

In [25]:
all['Variation'].value_counts(dropna=False)

Variation
Test       26959
Control    23526
Unknown    20106
Name: count, dtype: int64

In [32]:
# Divide the merged data frame into test, control and unknown groups

groups = all.groupby('Variation')

# Accessing groups and creating new DataFrames
test = groups.get_group('Test')
control = groups.get_group('Control')
unknown = groups.get_group('Unknown')

In [33]:
test['is_confirmed'].value_counts()

is_confirmed
True     18682
False     8277
Name: count, dtype: int64

In [34]:
control['is_confirmed'].value_counts()

is_confirmed
True     15428
False     8098
Name: count, dtype: int64

In [18]:
# sns.histplot(data=control, x='is_confirmed', bins=2)